# HW1 - Higher order functions and Typing
## https://www.cs.bgu.ac.il/~ppl172

Provide your answers in each cell of this notebook marked with **answer**.  
Whenever justifications are requested, write your explanation clearly and concisely.

When you write TypeScript code, make sure it passes compilation (tsc) with no type checking error.

All the places where answers are expected are marked with '@@@'.

# Q1 Theoretical Questions
### Q1.1 Type Relations 

* What is the intersection of the types denoted by the following two expressions:
  
  `{a:number}[]` and `{b:string}[]`
  
  Describe values in the type (give examples, characterize them) and write a TypeScript type expression that denotes this intersection.
  

### 1.1 Answer
{a:number, b:string}[]

Example:
let a: {a:number}[] = [{a:8}];
let b: {b:string}[] = [{b:'s'}];

let c: {a:number, b:string}[] = [{a:8, b:'s'}];

a=c; #compiled
b=c; #compiled

c=a; #compilation error: Type '{ a: number; }[]' is not assignable to type '{ a: number; b: string; }[]'. Type '{ a: number; }' is not assignable to type '{ a: number; b: string; }'. Property 'b' is missing in type '{ a: number; }'.

### Q1.2 Subtypes

* For each of the pairs (T1, T2) below, determine whether the type T1 is a subtype of T2 - justify your answer:

```
1.2.1 
type T1 = {a:number, b:{}}
type T2 = {a:number}

1.2.2
type T1 = {a:number, b:any}
type T2 = {a:number}

1.2.3
type T1 = {a:number, b:undefined}
type T2 = {a:number}
```

### 1.2 Answer

1.2.1
Yes, all the values in T1 satisfy the constraints of T2.

type T1 = {a:number, b:{}}
type T2 = {a:number}

let a: T1 = {a:8,  b:{a:1, b:"s", c:false}};
let b: T2 = {a:8};

b=a; #compiled

1.2.2
Yes, all the values in T1 satisfy the constraints of T2.

type T1 = {a:number, b:any}
type T2 = {a:number}

let a: T1 = {a:8,  b:true};
let b: T2 = {a:8};

b=a; #compiled

1.2.3
Yes, all the values in T1 satisfy the constraints of T2.

type T1 = {a:number, b:undefined}
type T2 = {a:number}

let a: T1 = {a:8,  b:undefined};
let b: T2 = {a:8};

b=a; #compiled


### Q1.3 Type Inference

* Write the type of the following values as a TypeScript type expression:
  * let v1 = `[{a:1}]`,
  *     v2 = `{a:[1]}`
  *     v3 = `x => x / 2`,
  *     v4 = `(f, x) => f(f(x))`

### 1.3 Answer

Type of v1 is: 
```
{a:number}[]
```

Type of v2 is:
```
{a:number[]}
```

Type of v3 is:
```
(x:number) => number
```

Type of v4 is:
```
<T>(f:<T>(x:T)=>T, x:T) => T
```

* Compare the expression in `v3` and `v4` and explain when the inferred type expression contains type variables.

**Answer:**

In v3 we have mathematical function thats why we understood that the type should be a number.
In v4 we use genric type and not 'any' beacuse the funcation gets the same value as it returns and operate it again.

### Q1.4 Function Type Inference

Consider the following function:

In [1]:
const invokeN1 = (f, n) => 
    n === 0 ? (x) => x : (x) => f(invokeN1(f,n-1)(x))

true

  * Describe what the function computes in words
  * Write the type of the function invokeN.

### 1.4 Answer

The function invokeN1 compute f, n times on input x.
For example, invokeN1(f,3)(x) === f(f(f(x)))

const invokeN1:<T>(f:<T>(x:T)=>T, n:Number) => <T>(x:T)=>T = (f, n) =>
    n === 0 ? (x) => x : (x) => f(invokeN1(f,n-1)(x))


### Q1.5 Types and Values

* **1.5.1**: Is it possible to design a type in TypeScript that denotes the empty set using the constructs described in class?

* **Answer 1.5.1**:

No. From typescript documentation: "By default null and undefined are subtypes of **all** other types. That means you can assign null and undefined to something like number."

* **1.5.2** Is it possible to design a type in TypeScript that denotes a finite set of 5 values?

* **Answer 1.5.2**:

No.

* **1.5.3** Is it possible to design a type in TypeScript that denotes the set of even numbers?

* **Answer 1.5.3**:

Possible. Because JavaScript has maximum & minimum numeric value (Number.MAX_VALUE, Number.MIN_VALUE), we can code: 
type even = ....-2|0|2|4|6....
it is a finite expression.

# Q2 Functional Abstractions over Trees

Consider the definition of the inductive Tree data type.

The set of Tree values is defined inductively as follows:
* There is no empty tree.
* The smallest tree is a tree with no children - a single node containing a single data element denoted $leaf(v)$.
* For a set of trees $t_1, \ldots, t_n$ and a data element $v_0$, one can construct a new tree whose root is has value $v_0$ and children $(t_1, \ldots, t_n)$ - denoted $tree(v_0, (t_1, \ldots, t_n))$.

In this inductive definition, the height of the trees determines the inductive size of trees. The height of a tree is defined inductively as:
* The height of a tree with no children is 0: $h(leaf(v)) = 0$.
* For a tree with children, the height of the tree is 1 more than the max height of the children:
  $h(tree(v_0, (t_1, \ldots, t_n))) = 1 + max_{i \in \{1 \ldots n\}} h(t_i)$
  
Other properties or computations on trees have a similar inductive definition.

We write the following definition of a type denoting the set of Tree values as follows:

In [2]:
interface Tree<T> {
    root: T;
    children: Tree<T>[];
}

{
    let b: Tree<number> = {
        root: 1,
        children: [
            {root: 2, children: []},
            {root: 3, children: [
                {root: 4, children:[]}
            ]}
        ]
    }
    b;
}

{ root: 1,
  children: [ { root: 2, children: [] }, { root: 3, children: [Object] } ] }

We define the following functions to manipulate Tree values - providing an abstract interface for trees.
It consists of two constructors - for leaf trees and non-leaf trees - and three accessors:

In [3]:
const makeLeaf : <T>(v:T)=>Tree<T> =
    v => {return {root: v, children:[]};}
    
const makeTree : <T>(v:T, children:Tree<T>[])=>Tree<T> =
    (v, children) => { return {root:v, children:children}}

const treeRoot : <T>(t: Tree<T>)=>T =
    t => t.root;

const treeChildren: <T>(t: Tree<T>)=>Tree<T>[] =
    t => t.children;
    
const treeLeaf : <T>(t: Tree<T>)=>boolean =
    t => t.children.length === 0;

true

In [4]:
treeLeaf(makeLeaf(5))

true

A function manipulating Tree values will typically perform a recursion similar to that observed in the definition of the `treeHeight` function:

In [5]:
import {map, reduce} from 'ramda'

const treeHeight : <T>(t:Tree<T>)=>number =
    t => treeLeaf(t) ? 
            0 : 
            1 + reduce(Math.max, 0, map(treeHeight, treeChildren(t)));
            
treeHeight(makeTree(1, [makeLeaf(2), makeLeaf(3)]))

1

In [6]:
treeHeight({ root: 1, children: [ { root: 2, children: [] }, { root: 3, children: [{root: 4, children:[]}] } ] })

2

Other functions that operate on tree values will adopt a similar pattern of recursion:

```
const f: <T1,T2>(t:Tree<T1>)=>T2 =
    t => treeLeaf(t) ? <basecase> : ...map(f, t.children)...;
```

In this question, we define **functional abstractions** over Tree values similar to the sequence interface we used over Array values - `map, filter, forEach, some, every, reduce`.

We first define the type of the functions that will be passed as arguments to these higher order functions:

In [7]:
type Command<T> = (x:T)=>void;
type Transformer<T1,T2> = (x:T1)=>T2;
type Predicate<T> = (x:T)=>boolean;
type Accumulator<T1,T2> = (acc:T1, item:T2)=>T1;

true

**Define the following functional abstractions over trees.**

For each function:
* provide the function definition 
* and four good test covering the code of the function as much as possible.

Make sure your solution follows Functional Programming practice with no mutation to any variable.

### Question 2.1 treeMap

`treeMap` operates over a Tree value.  It returns a new tree of the same shape as its parameter, where the value of each node is transformed by `f`.

In [8]:
// Answer 2.1
const treeMap: <T1,T2>(f: Transformer<T1,T2>, tree: Tree<T1>)=>Tree<T2> =
    (f, tree) => treeLeaf(tree) ?
            makeLeaf(f(treeRoot(tree))) :
            makeTree(f(treeRoot(tree)), treeChildren(tree).map((tree) => treeMap(f, tree)));

true

In [9]:
// treeMap test 1
import * as assert from 'assert';
let tree1 = makeTree(1, [makeLeaf(2), makeLeaf(3)]);
assert.deepEqual(treeMap((x:number) => x*x, tree1), { root: 1, children: [ { root: 4, children: [] }, { root: 9, children: [] } ] } , "test1");
"all ok";

'all ok'

In [10]:
// treeMap test 2
let tree2 = makeTree("a", [makeLeaf("aa"), makeLeaf("aaa")]);
assert.deepEqual(treeMap((s:string) => s.length , tree2), { root: 1, children: [ { root: 2, children: [] }, { root: 3, children: [] } ] } , "test2");
"all ok";

'all ok'

In [11]:
// treeMap test 3
let tree3 = makeTree("aaaa", []);
assert.deepEqual(treeMap((s:string) => s.length , tree3), { root: 4, children: [] } , "test3");
"all ok";

'all ok'

In [12]:
// treeMap test 4
let tree4 = makeTree(1,  [ makeTree(2, [makeLeaf(3), makeLeaf(4)]) , makeTree(5, [makeLeaf(6), makeLeaf(7)])  ]   );
let newTree = treeMap((x:number) => x*x, tree4)
assert.deepEqual(newTree.children[1].children[1].root, 49 , "test4");
"all ok";

'all ok'

### Question 2.2 treeForEachDF

`treeForEachDF` operates over a Tree value.  It traverses the tree node by node in depth-first order and applies the command `f` on each one.  The side effects of the command are observed in depth-first order.

In [13]:
// Answer 2.2 
const treeForEachDF: <T>(f: Command<T>, tree: Tree<T>)=>void =
    (f, tree) => {
              if (treeLeaf(tree)){
                f(treeRoot(tree));
              }else{
                treeChildren(tree).forEach((tree) => treeForEachDF(f, tree));
                f(treeRoot(tree));
              }
            }

true

In [14]:
// treeForEachDF test 1
let i=0
treeForEachDF( () => {++i}, makeTree(1,[]))
assert.ok(i === 1)
"all ok"

'all ok'

In [15]:
// treeForEachDF test 2
let tmp ="DFS - ";
treeForEachDF((x) => {tmp=tmp+x}, makeTree("d!!",[makeTree("y ", [makeLeaf("ve"), makeLeaf("r")]) , makeTree("o", [makeLeaf("g"), makeLeaf("o")])]))
assert.ok(tmp === "DFS - very good!!")
"all ok"

'all ok'

In [16]:
// treeForEachDF test 3
let j=2
treeForEachDF((x) => {j=j+x}, makeTree(1,[makeTree(2, [makeLeaf(3), makeLeaf(4)]) , makeTree(5, [makeLeaf(6), makeLeaf(7)])]))
assert.ok(j === 30)
"all ok"

'all ok'

In [17]:
// treeForEachDF test 4
let tree = makeTree(1,[makeTree(2, [makeLeaf(3), makeLeaf(4)]) , makeTree(5, [makeLeaf(6), makeLeaf(7)])])
let k=0
treeForEachDF( () => {++k}, tree)
assert.ok(k === 7)
"all ok"

'all ok'

### Question 2.3 treeReduceDF

`treeReduceDF` operates in a way similar to `reduce` on arrays: it traverses the tree in depth-first order, and at each node `t`, it computes `f(acc, t.root)` and continues with the value returned as the value of `acc` passed in the next node.

In [18]:
const treeReduceDF: <T1,T2>(f: Accumulator<T1,T2>, init: T1, tree: Tree<T2>)=>T1 =
  (f, init, tree) =>
        (treeLeaf(tree)?
          f (init ,(treeRoot(tree))):
          f (treeChildren(tree).reduce((x, tree) => treeReduceDF(f, x ,tree) , init) ,(treeRoot(tree))));

true

In [19]:
// treeReduceDF test 1
assert.ok(6 === treeReduceDF<number, number>((x, y)=>x+y, 0, makeTree(1, [makeLeaf(2), makeLeaf(3)])))
"all ok"

'all ok'

In [20]:
// treeReduceDF test 2
assert.ok("DFS - OK!" === treeReduceDF<string, string>((x, y)=>x+y, "DFS - ", makeTree("!", [makeLeaf("O"), makeLeaf("K")])))
"all ok"

'all ok'

In [21]:
// treeReduceDF test 3
assert.ok(128 === treeReduceDF<number, number>((x, y)=>x*y, 1,  makeTree(2,[makeTree(2, [makeLeaf(2), makeLeaf(2)]) , makeTree(2, [makeLeaf(2), makeLeaf(2)])])));
"all ok"

'all ok'

In [22]:
// treeReduceDF test 4
assert.ok(33343 === treeReduceDF<number, number>(Math.max, -1000, makeTree(-4,[makeTree(2525, [makeLeaf(44), makeLeaf(0)]) , makeTree(1212, [makeLeaf(33343), makeLeaf(5)])])));
"all ok"

'all ok'

### Question 2.4

Why do we need to specify the order of traversal of the tree for `treeReduceDF`?

Provide examples showing that with a different order of traversal, we could obtain different results when invoking `treeReduce`
with the same parameters.

### Answer 2.4
We need to specify the order of traversal of the tree for `treeReduceDF`beacuse if we don't we can get diferent results depending the function and the type to the nodes.
for example:

console.log(treeReduceDF<string, string>((x, y)=>x+y, 0, makeTree('a', [makeLeaf('b'), makeLeaf('c')])))
in DFS = > 'bca'

console.log(treeReduceDF<string, string>((x, y)=>x+y, 0, makeTree('a', [makeLeaf('b'), makeLeaf('c')])))
in BFS = > 'abc'

console.log(treeReduceDF<number, number>(Math.pow, 0, makeTree(1, [makeLeaf(2), makeLeaf(3)])))
in DFS = > 8

console.log(treeReduceDF<number, number>(Math.pow, 0, makeTree(1, [makeLeaf(2), makeLeaf(3)])))
in BFS = > 1


# Question 3: Runtime Type Checker

We have practiced with TypeScript - a compiler which operates over JavaScript programs enriched with type annotations in the TypeScript type language and which performs type checking at compilation time, and outputs a JavaScript program as output.

The TypeScript compiler (tsc) performs what is called **static code analysis** - an analysis of a program performed without executing the program itself.

In this question, we design a **runtime type checker** - a function that verifies the type of a value at runtime - that is, check **at runtime** that a value belongs to a type.

We call this function:

```
typeCheck(jsValue, typeExpression)=>boolean;
```

We first need to understand that this capability does **not** exist with TypeScript.
TypeScript is a compiler - it provides no runtime library, it does not exist at runtime.
After a TypeScript program is compiled into JavaScript, there are no types left in the JavaScript program, 
no type expressions and no type names.

The values we want to type check at runtime, typically, will be JSON values obtained by reading from a file or from an HTTP call to a server returning JSON values.

To achieve this goal, we need to represent type expressions as concrete values which will be passed to our function `typeCheck`.
Remember - at runtime, TypeScript type expressions do not exist - so we need to create our own type expressions.

### Question 3.1 Concrete Representation of Type Expressions

Let us consider a subset of the TypeScript type language that only includes the following type expressions:
* Atomic type expressions: `number`, `string`, `boolean`
* Arrays: for any type expression `T`, `T[]` is an array type expression.
* Maps: for any type expressions T1,...,Tn and any strings s1,...sn - `{s1:T1, ..., sn:Tn}` is a map type expression.

This is an inductive definition of the set of type expression values.

Compared to the TypeScript type system we discussed in class, we do not consider function types, type unions, intersection, generic types, recursive types, undefined, any and named types.

Our objective is to design a TypeScript type whose values are type expressions in this limited subset.
We will also define functions to operate over this data type (constructors and accessors).

For example, we want to define a value that represents the type expression `string[]`.
We will use the following value:

```
const teSA = {
    tag: "array",
    itemType: "string"
};
```

This value is the concrete representation of the type expression.
We can pass it as an argument to the `typeCheck` function as follows:

```
typeCheck(["aa", "bb"], teSA); // true
typeCheck(["aa", 1], teSA);    // false
```

We provide the following definitions using the acronym TE for Type Expression:

In [23]:
// We adopt the Disjoint Union type pattern discussed in class.

type TE = TeNumber | TeBoolean | TeString | TeArray | TeMap;

// Atomic type expressions have no component
interface TeNumber {
    tag: "number";
};
interface TeBoolean {
    tag: "boolean";
};
interface TeString {
    tag: "string";
};

// Concrete representation of an Array Type Expression
interface TeArray {
    tag: "array";
    itemType: TE;
};

true

**3.1** Complete the following type definition:

In [24]:
// ANSWER 3.1: Complete the type definition
interface TeMap {
    tag: "map",
    keys: String[],
    types: TE[]
};

true

### Question 3.2 Type Expression Constructors and Accessors

We define an abstraction barrier around the `TE` data type in the form of constructors for type expressions
of each type and accessors to retrieve the components of compound type expression values.

Complete the types and definitions of the functions marked by @@@:

In [25]:
// Constructors
// ============
const makeTeString : ()=>TeString = 
    () => { return {tag:"string"}}

const makeTeNumber : ()=>TeNumber =
    () => { return {tag:"number"}}

const makeTeBoolean : ()=>TeBoolean =
    () => { return {tag:"boolean"}}

const makeTeArray : (itemType: TE)=>TeArray =
    itemType => { return {tag: "array", itemType: itemType}}

const makeTeMap : (keys: String[], types: TE[])=>TeMap =
    (keys, types) => { return {tag: "map", keys: keys, types:types}}


// Type predicates
// ===============
const isTeNumber : (te:TE)=>boolean =
    te => te.tag === "number";

const isTeBoolean : (te:TE)=>boolean =
    te => te.tag === "boolean";

const isTeString : (te:TE)=>boolean =
    te => te.tag === "string";

const isTeArray : (te:TE)=>boolean =
    te => te.tag === "array";

const isTeMap : (te:TE)=>boolean =
    te => te.tag === "map";


// Accessors
// =========
const teArrayItemType : (tea : TeArray)=>TE =
    tea => tea.itemType;

const teMapKey : (tem : TeMap, key: string)=> TE =
    (tem, key) => tem.types[tem.keys.indexOf(key)]

true

Test these functions by constructing the value representing the following type expression and accessing its components:

```
{ a:string; b:number[]; c:{c1:boolean; c2:string[][]}}
```

In [26]:
// Test constructors
assert.deepEqual({tag:"string"}, makeTeString() , "test1");
assert.deepEqual({tag:"boolean"}, makeTeBoolean() , "test1");
assert.deepEqual({tag:"number"}, makeTeNumber() , "test1");
assert.deepEqual({tag:"array", itemType:{tag:"string"}}, makeTeArray(makeTeString()) , "test1");
assert.deepEqual({tag:"map", keys:["a"], types:[{tag:"string"}]}, makeTeMap(["a"],[makeTeString()]) , "test1");
"all ok"

let obj1 = makeTeMap(["a", "b", "c"],[makeTeString(), makeTeArray(makeTeNumber()), makeTeMap(["c1", "c2"],[makeTeBoolean(), makeTeArray(makeTeArray(makeTeString()))])])
// Test accessors
assert.deepEqual(teMapKey(obj1, "a"), makeTeString() , "test1");
assert.deepEqual(teMapKey(obj1, "b"), makeTeArray(makeTeNumber()) , "test1");
assert.deepEqual(teMapKey(obj1, "c"), makeTeMap(["c1", "c2"],[makeTeBoolean(), makeTeArray(makeTeArray(makeTeString()))]) , "test1");
assert.deepEqual(teMapKey(<TeMap>teMapKey(obj1, "c"), "c1"), makeTeBoolean() , "test1");
assert.deepEqual(teMapKey(<TeMap>teMapKey(obj1, "c"), "c2"), makeTeArray(makeTeArray(makeTeString())) , "test1");
"all ok"

'all ok'

### Question 3.3: typeChecker as a Recursive Algorithm

We now design the type checker algorithm.
Given a JavaScript value and a type expression value - we want to verify whether the value belongs to the type denoted by the type expression.

We design the rules of type checking in an inductive manner - through structural induction on the structure of the type expression values.

```
Type Checker Algorithm: 
typeCheck(jsValue:any, te:TE)=>boolean
Verify that jsValue belongs to the type denoted by te.

if te is atomic:
    if te is `number`: checkNumber(jsValue)
    else if te is `string`: checkString(jsValue)
    else if te is `boolean`: checkBoolean(jsValue)
else te is a compound type expression:    
    if te is an array: checkArray(jsValue, te)
    if te is a map: checkMap(jsValue, te)
```

We know that this algorithm tests all possible values of the `te` parameter based on its type (a disjoint union type).

We now turn to the definition of the sub-procedures in this definition:

For atomic type expressions - we rely on the JavaScript introspection primitives - `typeof`:
```
checkNumber: 
checkNumber(jsValue:any)=>boolean:
Verify that jsValue belongs to the set of numbers in JavaScript.

    typeof jsValue === "number"
```

#### Answer 3.3.1

Complete the algorithm definition (using the same pseudo code format as the definitions above):

```
checkString:
checkString(jsValue:any)=>boolean:
Verify that jsValue belongs to the set of strings in JavaScript.

    typeof jsValue === "string"

checkBoolean:
checkBoolean(jsValue:any)=>boolean:
Verify that jsValue belongs to the set of booleans in JavaScript.

    typeof jsValue === "boolean"

```

For compound type expressions - we need to check the JavaScript type of jsValue, verify what type of container it is (array or map), and if it matches the type denoted by `te` (array or map respectively) then, recursively, type check the components of the value.

Note that in the recursive step, we break down jsValue using the JavaScript accessors of the JavaScript value container, and we break down the `te` type expression using the accessors of the Type Expression data type.

For Array type expressions - this recursive method is defined as follows:
```
checkArray:
checkArray(jsValue:any, te:TeArray)=>boolean:
Verify the jsValue belongs to the set of array values denoted by te.

    jsValue instanceof "Array" && every item in jsValue verifies
        checkType(item, te.itemType)
```

#### Answer 3.3.2

Complete the algorithm definition of:
* checkMap

checkMap:
checkMap(jsValue:any, te:TeMap)=>boolean:
Verify the jsValue belongs to the set of map values denoted by te.

    jsValue instanceof "Object" && every value in jsValue verifies
        checkType(jsValue.key, te.types[te.keys.indexOf(key)])


### Question 3.4 Implement the typeCheck function

Implement in TypeScript the recursive typeCheck algorithm defined above.
Use the `switch` method to manipulate disjoint union types as discussed in class.

In [27]:
// Answer 3.4
const checkNumber: (jsValue:any)=>boolean =
    (jsValue) => typeof jsValue === "number"

const checkString: (jsValue:any)=>boolean =
    (jsValue) => typeof jsValue === "string"

const checkBoolean: (jsValue:any)=>boolean =
    (jsValue) => typeof jsValue === "boolean"

const checkArray: (jsValue:any, te:TeArray)=>boolean =
   (jsValue, te) =>  jsValue instanceof Array && jsValue.map( (childJsValue) => typeCheck(childJsValue, teArrayItemType(te))).reduce ((x, y)=>x&&y, true)

const checkMap: (jsValue:any, te:TeMap)=>boolean =
    (jsValue, te) => jsValue instanceof Object && Object.keys(jsValue).map( (key) => typeCheck(jsValue[key], teMapKey(te, key))).reduce ((x, y)=>x&&y, true)

const typeCheck : (jsValue:any, te:TE)=>boolean =
    (jsValue, te) => {
      switch (true){
        case (te===undefined): return false;
        case isTeNumber(te): return checkNumber(jsValue);
        case isTeString(te): return checkString(jsValue);
        case isTeBoolean(te): return checkBoolean(jsValue);
        case isTeArray(te): return checkArray(jsValue, <TeArray>te);
        case isTeMap(te): return checkMap(jsValue, <TeMap>te);
        default: return false;
      }
    }

true

In [28]:
// Test typeCheck with 10 good tests covering the code in increasing order of complexity.
// Make sure to test positive and negative cases (cases where typeCheck returns true and false).
// Test 0
assert.ok(true === typeCheck("test", makeTeString()));
"all ok"

'all ok'

In [29]:
// Test 1
assert.ok(true === typeCheck(true, makeTeBoolean()));
"all ok"

'all ok'

In [30]:
// Test 2
assert.ok(true === typeCheck(9, makeTeNumber()));
"all ok"

'all ok'

In [31]:
// Test 3
assert.ok(false === typeCheck(false, makeTeString()));
"all ok"

'all ok'

In [32]:
// Test 4
assert.ok(true === typeCheck([1,2,3], makeTeArray(makeTeNumber())));
"all ok"

'all ok'

In [33]:
// Test 5
assert.ok(false === typeCheck([1,2,3], makeTeArray(makeTeString())));
"all ok"

'all ok'

In [34]:
// Test 6
assert.ok(true === typeCheck([[90],[80],[90]], makeTeArray(makeTeArray(makeTeNumber()))));
"all ok"

'all ok'

In [35]:
// Test 7
assert.ok(true === typeCheck({a:8, b:"8"}, makeTeMap(["a", "b"],[makeTeNumber(), makeTeString()])));
"all ok"

'all ok'

In [36]:
// Test 8
assert.ok(false === typeCheck({a:8, c:"8"}, makeTeMap(["a", "b"],[makeTeNumber(), makeTeString()])));
"all ok"

'all ok'

In [37]:
// Test 9
// { a:string; b:number[]; c:{c1:boolean; c2:string[][]}}
let obj2 = makeTeMap(["a", "b", "c"],[makeTeString(), makeTeArray(makeTeNumber()), makeTeMap(["c1", "c2"],[makeTeBoolean(), makeTeArray(makeTeArray(makeTeString()))])])
assert.ok(true === typeCheck({a:"123", b:[1,2,3], c:{c1:false, c2:[["o","k"],["g","o","o","d"]]}}, obj2));
"all ok"

'all ok'